# Predição de Casos de Covid-19 de Acordo com o Nº de Casos e Critérios Socioeconômicos

## Preprocessamento dos Dados

In [7]:
import pandas as pd 
import numpy as np 

## Ler arquivo csv e armazerna em formato DataFrame
dados_covid = pd.read_csv('municipios_RN_COVID19.csv')
dados_socioeconomico = pd.read_csv('AtlasBrasil_IndicadoresSociais.csv', sep=';')

## Limpa os dados faltantes do dataset
dados_covid = dados_covid.dropna()

In [8]:
## Padroniza todas as cidades com nome minúsculo
dados_covid['city'] = dados_covid['city'].str.lower()
dados_socioeconomico['Espacialidades'] = dados_socioeconomico['Espacialidades'].str.lower()

In [9]:
## Cria e insere colunas no dataframe que será utilizado para a predição dos dados
dados_covid_socioeconomico = pd.DataFrame()
dados_covid_socioeconomico['date'] = dados_covid.date
dados_covid_socioeconomico['city'] = dados_covid.city
dados_covid_socioeconomico['last_available_confirmed'] = dados_covid.last_available_confirmed
dados_covid_socioeconomico['Gini_Index_2010'] = np.nan
dados_covid_socioeconomico['access_to_water_and_basic_sanitatio_per_cent'] = np.nan
dados_covid_socioeconomico['MHDI_income'] = np.nan
dados_covid_socioeconomico['MHDI_education'] = np.nan
dados_covid_socioeconomico['aging_rate'] = np.nan
dados_covid_socioeconomico['extremely_poor_per_cent'] = np.nan
## Coloca os dados socioeconomicos em cada célula do dataframe de acordo com a cidade
for municipio in dados_covid_socioeconomico.city:
    for index in dados_socioeconomico.Espacialidades:
        if(municipio == index):
            dados_covid_socioeconomico.loc[dados_covid_socioeconomico.city == index,'Gini_Index_2010'] = dados_socioeconomico.loc[dados_socioeconomico.Espacialidades == municipio, 'Indice_de_Gini_2010'].item()
            dados_covid_socioeconomico.loc[dados_covid_socioeconomico.city == index,'access_to_water_and_basic_sanitatio_per_cent'] = dados_socioeconomico.loc[dados_socioeconomico.Espacialidades == municipio, 'pessoas em domicílios com abastecimento de água e esgotamento sanitário inadequados 2010'].item()
            dados_covid_socioeconomico.loc[dados_covid_socioeconomico.city == index,'MHDI_income'] = dados_socioeconomico.loc[dados_socioeconomico.Espacialidades == municipio, 'IDHM Renda 2010'].item()
            dados_covid_socioeconomico.loc[dados_covid_socioeconomico.city == index,'MHDI_education'] = dados_socioeconomico.loc[dados_socioeconomico.Espacialidades == municipio, 'IDHM Educação 2010'].item()
            dados_covid_socioeconomico.loc[dados_covid_socioeconomico.city == index,'aging_rate'] = dados_socioeconomico.loc[dados_socioeconomico.Espacialidades == municipio, 'Taxa de envelhecimento 2010'].item()
            dados_covid_socioeconomico.loc[dados_covid_socioeconomico.city == index,'extremely_poor_per_cent'] = dados_socioeconomico.loc[dados_socioeconomico.Espacialidades == municipio, 'extremamente pobres 2010'].item()

## Converte as primeiras letras para maiúsculo
dados_covid_socioeconomico['city'] = dados_covid_socioeconomico['city'].str.title()

Analisando os dados contidos no DataFrame dados_covid_socioeconomico, podemos perceber que Serra Caiada não possui um Índice de Gini. Assim, iremos remover o município do nosso DataFrame. 

In [10]:
## Limpa os dados faltantes do dataframe
dados_covid_socioeconomico = dados_covid_socioeconomico.dropna()

dados_covid_socioeconomico.to_csv(path_or_buf='Dados_Covid_Socio.csv', index=False)

## Selecionando e separando o X e Y

In [5]:
X = dados_covid_socioeconomico.iloc[:, [0, 1]].values
y = dados_covid_socioeconomico.iloc[:, 2].values

### Modelo de Regressão Polinomial

---

O primeiro teste será feito utilizando apenas o número de casos, a data, a cidade e o índice de Gini

In [24]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
## Realiza a tranformação de variáveis Categóricas de X em variáveis numéricas 
labelencoder_x = LabelEncoder()
for index in range(len(X[0])):
    X[:, index] = labelencoder_x.fit_transform(X[:, index])

In [25]:
from sklearn.compose import ColumnTransformer
## Categoriza as variáveis Nominais (Raça, Sexo, Profissão, Religião, Cidade, ...)
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 1])], remainder='passthrough')
X = onehotencorder.fit_transform(X).toarray()

In [26]:
from sklearn.preprocessing import StandardScaler
## Padroniza os valores
scaler_x = StandardScaler()
X = scaler_x.fit_transform(X)
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y.reshape(-1,1))

In [ ]:
from sklearn.model_selection import train_test_split
# Separa as amostras de Teste e Treinamento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
# Torna a variável X_train e X_test exponencial
poly = PolynomialFeatures(degree=4)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.fit_transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
## Treina o regressor
regressor_linear = LinearRegression()
regressor_linear.fit(X_poly_train, y_train) 

In [ ]:
## Realiza predição
pred_poly = regressor_linear.predict(X_poly_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_test, pred_poly)
coef = r2_score(y_test, pred_poly)

print('Mean Absolute Error: ', mae)
print('Coefficient of Determination: ', coef)

In [ ]:
from sklearn.model_selection import cross_val_score

results_poly = cross_val_score(regressor_linear, X, y, cv=10)
print('Cross Validation Mean Score: ', results_poly.mean())

### Modelo de Regressão com Random Forest

---


In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor_rforest = RandomForestRegressor(n_estimators=100, criterion='mse')
regressor_rforest.fit(X_train, y_train)

In [ ]:
regressor_rforest.score(X_train, y_train)

In [ ]:
pred_rfor = regressor_rforest.predict(X_test)

mae = mean_absolute_error(y_test, pred_rfor)
coef = r2_score(y_test, pred_rfor)

print('Mean Absolute Error: ', mae)
print('Coefficient of Determination: ', coef)

In [36]:
results_rfor = cross_val_score(regressor_rforest, X, y, cv=10)
print('Cross Validation Mean Score: ', results_rfor.mean())

KeyboardInterrupt: 

### Modelo de Regressão com Redes Neurais

---

In [ ]:
'''X = dados_covid_socioeconomico.iloc[:, [0, 1, 3]].values
y = dados_covid_socioeconomico.iloc[:, 2].values

for index in range(len(X[0])-1):
    X[:, index] = labelencoder_x.fit_transform(X[:, index])

X = scaler_x.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1,1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
'''
from sklearn.neural_network import MLPRegressor
regressor_mlp = MLPRegressor(hidden_layer_sizes=(16,16,16), activation='relu', solver='adam', learning_rate_init=0.0001)
regressor_mlp.fit(X_train, y_train)

In [ ]:
score_train = regressor_mlp.score(X_train, y_train)
score_train

In [ ]:
score_test = regressor_mlp.score(X_test, y_test)
score_test

In [ ]:
from sklearn.model_selection import cross_val_score

results_mlp = cross_val_score(regressor_mlp, X, y, cv=100)
print('Cross Validation Mean Score: ', results_mlp.mean())